# Mandatory Exercise - Session 7

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is toread all the pairs of sentences, compute their similarities using words + Nammed Enntities (NEs) and Jaccard Coefficient and show the results. 

In [ ]:
# initial imports. Could also be done in the PC
import nltk
import nltk.corpus
nltk.download('maxent_ne_chunker')
nltk.download('words')

#additional set of imports
from nltk import pos_tag
from nltk.metrics import jaccard_distance
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.parse import CoreNLPParser
from scipy.stats import pearsonr

# Core Named-entity tagger as stanford one is deprecated: https://github.com/nltk/nltk/issues/2010
tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')

# launch the server in terminal
# java -mx4g -cp "/home/jan/Downloads/stanford-corenlp-full-2018-10-05/*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

### 1.

In [ ]:
def get_stanford_named_entity_chunked(sentence):
    """Given the passed sentence string, returns an array with the chunks (words and named entities) it contains, using Stanford NLP"""
        
    # obtain an array with the sentence tokens
    tokenized_s = nltk.word_tokenize(sentence)
    
    # tag and run as a normal word or a named entity (e.g. a person or an organization)
    tagged_s = tagger.tag(tokenized_s)
    
    chunked_sentence = []
    last_token = ''
    last_tag = ''
    
    for tagged_token in tagged_s:
        
        token = tagged_token[0]
        tag = tagged_token[1]
        
        # make normal words have lower case, also discard punctuation marks
        if tag == 'O':
            if token.isalnum():
                chunked_sentence.append(token.lower())
         
        # keep named entities with the original capitalization
        else:
            if last_tag == tag:
                chunked_sentence[-1] += ' ' + token
            else:
                chunked_sentence.append(token)
        
        last_token = token
        last_tag = tag
    
    return chunked_sentence

# example: note it does not group the terms of named entities, always 1 by 1. 
print(get_stanford_named_entity_chunked("Mark Pedersen and John Smith are working at Google since 1994 for 1000$ per week."))

### 2. 

In [ ]:
def get_ne_chunked(sentence):
    """Given the passed sentence string, returns an array with the chunks (words and named entities) it contains, using NLTK"""
        
    # obtain an array with the sentence tokens
    tokenized_s = nltk.word_tokenize(sentence)
        
    # chunk and tag with NLTK named-entity tagged
    x = pos_tag(word_tokenize(sentence))
    chunk_tree = ne_chunk(x, binary=True)
    
    chunked_sentence = []
    for chunk in chunk_tree:
        
        # keep named entities with the original capitalization
        if hasattr(chunk, 'label'):
            token = ' '.join(term[0] for term in chunk)
            chunked_sentence.append(token)
            
        # make normal words have lower case, also discard puntuaction marks
        else:
            token = chunk[0]
            if token.isalnum():
                chunked_sentence.append(token.lower())
    
    return chunked_sentence

# example: note that NLTK naming-entity tagger does group the terms of named entities in a single string
print(get_ne_chunked("Mark Pedersen and John Smith are working at Google since 1994 for 1000$ per week."))

We now proceed to open and read the input file

In [ ]:
import os
import sys

# full path of the corpus file, with that the trial folder containing the input file being in the same directory as the JPN
absolute_file_path = os.path.dirname(os.path.abspath("__file__")) + "/./trial//STS.input.txt"

#value initialization and instantiation
d = {}
tests = []
standard = []

# find all sentence pairs in the document
sentence_pairs = []
sentence_set_pairs = []
with open(absolute_file_path) as f:
    lines = f.readlines()
    for line in lines:
        index, sentence0, sentence1 = line.split("\t")
        if index in d:
            d[index] = sentence0, sentence1
        else:
            d[index] = (get_stanford_named_entity_chunked(sentence0), get_stanford_named_entity_chunked(sentence1))
            print("First sentence: \t", sentence0, "\nSecond sentence: \t", sentence1, "\n")
    print()  
    
# double checking the format
for key in d:
    print("{}\n{}\n\n".format(d[key][0], d[key][1]))

### 2. Sentence similarity calculation and comparison with the gold standard.
The pairs of sentences are checked to see how similar they are, using the Jaccard distance. Previously the sentences must have been tokenized and we have picked the **Sets**, *unique values of those tokenized sentences*; The more words or named entities two sentences have in common, the more similar they are. Then we calculate the similarity as 1-JD. 

We compute the Jaccard distance. In this step, we must first tokenize the sentences. We then take the **sets**; *unique values of those tokenized sentences*, lemmatize them and compute the **jaccard similarity as 1 - jaccard distance**.

In [ ]:
for key in d:
        
    w1 = set(d[key][0])
    w2 = set(d[key][1])

    # jaccard similarity 1 - jaccard distance
    dist = jaccard_distance(w1, w2)
    jaccard_similarity = 1 - dist
    tests.append(round(jaccard_similarity,3))
print(tests)

## 3. Compare the results with those in session 2 (document) and 3 (morphology) in which words and lemmas were considered.
The gold standard results would be 5,4,3,2,1,0.


In session 2 we got the following results:

* *'id1': 0.308* --> 1.54
* *'id2': 0.263* --> 1.31
* *'id3': 0.467* --> 2.34
* *'id4': 0.455* --> 2.27
* *'id5': 0.231* --> 1.15
* *'id6': 0.138* --> 0.69

In session 3, using lemmas similarity, we obtained:
* *'id1': 0.333* --> 1.67
* *'id2': 0.412* --> 2.06
* *'id3': 0.571* --> 2.86
* *'id4': 0.333* --> 1.67
* *'id5': 0.167* --> 0.84
* *'id6': 0.138* --> 0.69

In session 6, using lesk, we obtained:
* *'id1': 0.333* --> 1.67
* *'id2': 0.375* --> 1.88
* *'id3': 0.571* --> 2.86
* *'id4': 0.273* --> 1.36
* *'id5': 0.167* --> 0.84
* *'id6': 0.103* --> 0.52

And our current result is:
* *'id1': 0.273* --> 1.37
* *'id2': 0.154* --> 0.77
* *'id3': 0.636* --> 3.18
* *'id4': 0.400* --> 2.00
* *'id5': 0.091* --> 0.09
* *'id6': 0.107* --> 0.53

Overall, the first and second result decreased -which is a pitty- since we managed to significantly imporve our results on id3 and id4. 



## 4. Compare the results with gold standard by giving the pearson correlation between them.
And now, we open the Golden Standard file and calculate the perason correlation with and without lemmatization. 

**Pearson Correlation**
It shows the linear relationship between two sets of data. That means: the strength of the association between the two variables. It has a value between +1 and −1, where 1 is total positive linear correlation, 0 is no linear correlation, and −1 is total negative linear correlation

**Coefficient Value** -- Strength of Association
0.1 < | r | < .3 -- small correlation
0.3 < | r | < .5 -- medium/moderate correlation
| r | > .5 -- large/strong correlation

In [ ]:
for line in open('./trial/STS.gs.txt','r'):
    line = line.strip().split("\t")
    standard.append(int(line[1]))

a = pearsonr(standard[0:6], tests[0:6])[0]
print('Pearson correlation:', round(a,3))

## 5. Conclusions

Over the development of this lab, we have computed the similarities between different pairs of sentences; calculated their jaccard distance.

We have also compared them with the gold standard and have achieved lower results than in the previous labs:
- Pearson correlation without lemmatization: 0.396
- Pearson correlation with lemmatization: 0.578
- Pearson correlation with lesk: 0.668
- **Pearson correlation using nammed entities: 0.318**

**However, it is important to note that this results would be more accurate that the one we obtained in S2, S3 and S6 since we removed the punctuation**. Performance has decreased but are now focusing in what actually matters. 

What we can already see is that in order to tweak the performance of each algorithm, we will have to sort out their order and application so that we obtain the best possible results for the project. 